# 과제: 네이버 영화 정보 및 평점 크롤링

- 대상: 예매순 상위 5개의 현재 상영 중인 영화
- 수집할 항목: 영화 제목, 주연배우 3인, 네티즌 평점, 관람객 평점, 기자/평론가 평점, 관람객 별점 리뷰 20건 공감순으로(평점, 작성자닉네임, 리뷰본문)

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

### 1. 예매순 상위 5개의 현재 상영 중인 영화 가져오기

영화 제목, 주연배우 3인

In [2]:
def movie_title_url_actor():
    url = 'https://movie.naver.com/movie/running/current.nhn'
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    
    #제목 추출
    titles = soup.select('dt[class="tit"] a', limit = 5)
    title = []
    for p in titles:
        title.append(p.get_text())
    
    #주연배우 3인 추출
    actors = []
    for i in range(2, 15, 3):
        actors.append(soup.select('span[class="link_txt"]')[i].select('a', limit=3))

    actors_list = []
    for i in range(len(actors)):
        temp = []
        if actors[i] == 3:
            for j in range(len(actors[i])):
                temp.append(actors[i][j].get_text())
            actors_list.append(temp)
        else: #주연배우가 3인 미만인 경우를 위한 예외처리 (나머지를 None으로 채운다)
            for j in range(len(actors[i])):
                temp.append(actors[i][j].get_text())
            num = 3 - len(actors[i])
            for n in range(num):
                temp.append("None")
            actors_list.append(temp)
            
    #합치기
    movie_actor = []
    for i in range(len(title)):
        temp = []
        temp.append(title[i])
        for j in range(len(actors_list[i])):
            temp.append(actors_list[i][j])
        movie_actor.append(temp)
        
    return movie_actor

### 2. 해당 영화의 평점 가져오기

네티즌 평점, 관람객 평점, 기자/평론가 평점

In [3]:
def get_grade(movie_actor):
    
    url = 'https://movie.naver.com/movie/running/current.nhn'
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')    
    
    #전달 받은 movie_actor에서 영화 제목만 추출한 titles 변수 새로 생성
    titles = []
    for i in range(len(movie_actor)):
        titles.append(movie_actor[i][0])
        
    #movie_actor로 전달받은 영화 제목의 url 가져오기
    movies = soup.select('dt[class="tit"] a')
    m_url = []
    for i in range(len(movies)):
        if movies[i].get_text() in titles:
            url = movies[i].attrs['href']
            m_url.append("http://movie.naver.com" + url)

#     print(m_url)
    
    #각각의 스코어 계산
    movie_scores = []
    for m in range(len(m_url)):
        url = m_url[m]
        res = requests.get(url)
        html = res.text
        soup = BeautifulSoup(html)

        score = soup.select('div[class="star_score"] em', limit = 12)

        scores = []
        temp = ''
        for i in range(len(score)):
            temp += score[i].get_text()
            if i in list(range(3, 12, 4)):
                scores.append(temp)
                temp = ''
        movie_scores.append(scores)

    return movie_scores

### 3. 관람객 평점 공감순 20건 가져오기

평점, 평점 작성자 닉네임, 리뷰 본문

In [4]:
def get_reviews(movie_actor):
    
    url = 'https://movie.naver.com/movie/running/current.nhn'
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')    
    movies = soup.select('dt[class="tit"] a')

    #전달 받은 movie_actor에서 영화 제목만 추출한 titles 변수 새로 생성
    titles = []
    for i in range(len(movie_actor)):
        titles.append(movie_actor[i][0])
    
    #titles로 전달받은 영화 제목의 code값 가져오기
    code = []
    for i in range(len(movies)):
        if movies[i].get_text() in titles:
            url = movies[i].attrs['href']
            code.append(url[-6:])
    
    total_review = []
    for i in range(len(titles)):
        #해당 영화의 url을 받아와 초기화
#         print("\n<", titles[i], ">")
        urls = []
        for j in range(1, 3):
            url = "https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=" + code[i] + "&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=" + str(j)
            urls.append(url)
        
        score_set = []
        id_set = []
        reviews_set= []
        review_20 = []

        for page in range(len(urls)):
            #순서대로 첫 번째 페이지, 두 번째 페이지를 돈다.
            url = urls[page]
            res = requests.get(url)
            html = res.text
            soup = BeautifulSoup(html)
            
            #평점 추출
            score = soup.select('div[class="star_score"] em')
            for s in range(len(score)):
                score_set.append(score[s].get_text())
            
            #작성자 추출
            user_id = soup.select('div[class="score_reple"] span')
            p = re.compile('\w*\s*\w*\(*\w*[*]{4}\)*')

            for id in range(len(user_id)):
                m = p.match(user_id[id].get_text().strip())

                if m:
                    id_set.append(m.group())

            #한줄평 추출
            reviews = []
            for r1 in range(10):
                reviews.append(soup.find_all('span', id="_filtered_ment_"+str(r1)))

            for r2 in range(len(reviews)):
                reviews_set.append(reviews[r2][0].get_text().strip())
            
            
        # 영화 하나에 대해 다 돌았다면, 각 set을 합쳐 total_review의 한 행으로 만듦.
        temp = []
        temp.append(score_set)
        temp.append(id_set)
        temp.append(reviews_set)
        total_review.append(temp)
        
#         print("\n ID : \n", id_set)
#         print('\n SCORE : \n', score_set)
#         print("\n REVIEW : \n", reviews_set)
#     print("\n TOTAL RESULT : \n", total_review)

    return total_review


### 4. 저장하기

In [5]:
def insert_grades(data, scores):
    movie_actor = pd.DataFrame.from_records(data, columns=['Title', 'actor1', 'actor2', 'actor3'])    #처음에 구한 movie_actor을 데이터프레임으로 만들고
    scores = pd.DataFrame.from_records(scores, columns=['aud_score', 'critic_score', 'netizen_score']) #두 번째로 구한 score도 데이터 프레임으로 만든 후에
    movie_score = pd.concat([movie_actor, scores], axis=1) #두 개를 합친다.

    return movie_score

In [6]:
def insert_reviews(data, reviews):
    reviews = pd.DataFrame.from_records(reviews, columns = ['review_score', 'writer', 'comment']) # 리스트 자체가 요소로 들어가도록 데이터 프레임을 구성한 후에
    movie_reviews = pd.concat([data, reviews], axis=1) #합친다.
    return movie_reviews

In [7]:
def save(data):
    # 저장 방식 : 데이터 셋을 두 개로 나눈다. 
    #첫 번째 데이터 셋 : 영화의 가장 바탕이 되는 정보를 저장한 데이터 셋. 영화의 제목과 주연배우, 세 가지 스코어에 대한 정보를 저장한다. 
    #두 번째 데이터 셋 : 영화에 대한 리뷰를 저장한 데이터 셋. 해당 영화의 리뷰 20건을 가져오되, 작성자, 평점, 리뷰내용을 가져온다. 
    
    # 그냥 리뷰를 리스트의 형태로 저장하는 방식도 고려하였으나 이 경우 코멘트에 대한 분석이 필요할 경우 데이터 프레임 안의 리스트 요소에 대해 접근한다는 번거로움과 
    # 가시성이 떨어진다는 단점이 있다고 생각하여 데이터 셋을 나누기로 하였습니다.
    
    # 첫 번째 데이터 셋에 movie_key(기본키)를 만들어 두 번째 데이터 셋에 외래키로 전달함으로써 참조할 수 있게 하였고
    # 두 번째 데이터 셋에는 movie_key와 review_num(리뷰순서 1~20)의 조합이 기본키가 될 수 있게 하였습니다. 
    
    data["movie_key"] = range(1,6)
    data = pd.DataFrame(data, columns = ['movie_key', 'Title', 'actor1', 'actor2', 'actor3', 'aud_score', 'critic_score', 'netizen_score', 'review_score', 'writer', 'comment'])
    
    reviews = []
    for i in range(len(data['Title'])):
        for j in range(len(data['review_score'][0])):
            re = []
            re.append(i+1)
            re.append(j+1)
            re.append(data['review_score'][i][j])
            re.append(data['writer'][i][j])
            re.append(data['comment'][i][j])
            reviews.append(re)

    reviews_df = pd.DataFrame.from_records(reviews, columns = ['movie_key', 'review_num', 'review_score', 'writer', 'comment'])
    data = data.drop(['review_score', 'writer', 'comment'], axis=1)
    return data, reviews_df

### 5. 크롤링하기

In [8]:
data = movie_title_url_actor()

In [9]:
scores = get_grade(data)

In [10]:
reviews = get_reviews(data)

In [11]:
data = insert_grades(data, scores)

In [12]:
data = insert_reviews(data, reviews)

In [13]:
movies, reviews = save(data)

In [14]:
movies

,movie_key,Title,actor1,actor2,actor3,aud_score,critic_score,netizen_score
0,1,지푸라기라도 잡고 싶은 짐승들,전도연,정우성,배성우,8.61,6.71,7.13
1,2,정직한 후보,라미란,김무열,나문희,8.71,5.38,7.73
2,3,1917,조지 맥케이,딘-찰스 채프먼,None,9.35,7.67,9.03
3,4,작은 아씨들,시얼샤 로넌,엠마 왓슨,플로렌스 퓨,9.24,8.00,8.90
4,5,클로젯,하정우,김남길,허율,8.41,5.50,6.87


In [15]:
reviews

,movie_key,review_num,review_score,writer,comment
0,1,1,10,bohemian(mabu****),"난 전도연의 화류계 캐릭터가 좋다. 무뢰한, 너는 내 운명, 카운트다운...그리고 ..."
1,1,2,10,최정규(cjg4****),전도연 연기 진짜 오진다...와 이 영화에서 완전 섹시하게 나온다 역시 명불허전임...
2,1,3,1,어쩌라고(dpfk****),아니 개봉당일날 9시 땡하고 부터 평점 쏟아지는게 말이 돼냐? 요즘 조조는 꼭두새벽...
3,1,4,10,달다(fxko****),8명의 배우가 모두 주인공 같은 느낌.
4,1,5,9,써니(tlag****),개존잼 역시 전도연이죠? 카리스마 미쳐벌여ㅠㅁㅠ
...,...,...,...,...,...
95,5,16,1,지애(maln****),최근에 본 것 중에 제일 최악..
96,5,17,10,삡(jiny****),김남길 눈빛이 다했다..재밌어요
97,5,18,10,maimai(enma****),진짜 심장 쫄깃하고 무서움 ㄷㄷㄷ 이건 극장에서 봐야함
98,5,19,10,난꽃이다(play****),무서운거 진짜 못보는대 혼자 봤거든요 걱정하시는분들 그걸 감안하고라도 보시는거 추천...
